In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
import random

In [ ]:
import json
import copy

In [ ]:
import time
import datetime

In [ ]:
from collections import Counter

# Prepare the data structure 

In [ ]:
# Slot = Index , Day , Hour
class SlotProcessor:
    def __init__(self, inputString):
        stringParts = inputString.split(",")
        cursorPosition = 0
        self.id = stringParts[cursorPosition].strip()
        cursorPosition += 1
        self.day = stringParts[cursorPosition].strip()
        cursorPosition += 1
        self.time = stringParts[cursorPosition].strip()
    """        
    def getSlotInfo(self):
        print('Index: {} | Day: {} | Hour:{}'.format(self.id, self.day, self.time))
    """

In [ ]:
# Room = Index , Name , Capacity
class RoomProcessor:
    def __init__(self, inputString):
        stringParts = inputString.split(",")
        cursorPosition = 0

        self.id = stringParts[cursorPosition].strip()
        cursorPosition += 1

        self.name = stringParts[cursorPosition].strip()
    """
    def getRoomInfo(self):
        print('Index: {} | name: {}'.format(self.id, self.name))
    """

In [ ]:
#  Course = Index , Name , Capacity , Arrangement , InstructorIndex
class CourseProcessor:
    def __init__(self, inputString):
        stringParts = inputString.split(",")
        cursorPosition = 0

        self.id = stringParts[cursorPosition].strip()
        cursorPosition += 1

        self.name = stringParts[cursorPosition].strip()
        cursorPosition += 1

        self.capacity = stringParts[cursorPosition].strip()
        cursorPosition += 1

        self.arrangement = stringParts[cursorPosition].strip()
        cursorPosition += 1

        self.instructorIndex = int(stringParts[cursorPosition].strip())
   
    def getCourseInfo(self):
        print("Course ID: {} | Course Name: {} | Roomtype: {} | Arrangement: {} | InstructorIndex: {}"\
              .format(self.id, self.name, self.capacity, self.arrangement, self.instructorIndex))
        
    def getCourseName(self):
        return self.name
    def getCourseInstructorIndex(self):
        return int(self.instructorIndex)
     

In [ ]:
# Instructor = Index , Name 
class InstructorProcessor:
    def __init__(self, inputString):
        stringParts = inputString.split(",")
        cursorPosition = 0

        self.id = stringParts[cursorPosition].strip()
        cursorPosition += 1

        self.name = stringParts[cursorPosition].strip()
        cursorPosition += 1
     
    def getInstructorIndex(self):
        return int(self.id)
    
    def getInstructorName(self):
        return self.name
    
    def getInstructorInfo(self):
        print("Instructor ID: {} | Instructor Name: {}".format(self.id, self.name))
    
    # Highlight function getInstructorNameViaIndex(instructorIndex)
    def getInstructorNameViaIndex(self, instructorIndex):
        if int(self.id) == instructorIndex:
            print (self.name) 
        else:
            print('There is no instructor with this instructor index')


In [ ]:
slots = []
rooms = []
courses = []
instructors = []
with open('input4.txt') as fp:
    for line in fp:
        if line.startswith("#"):
        # ignore comments in input file
            continue
        lineSections = line.split("=")
        # print(lineSections)
        if lineSections[0].strip() == "Slot":
            #1 , one    ,  8:00
            # print(lineSections[1])
            slots.append(SlotProcessor(lineSections[1]))
            # slots
        elif lineSections[0].strip() == "Room":
            rooms.append(RoomProcessor(lineSections[1]))
        elif lineSections[0].strip() == "Course":
            courses.append(CourseProcessor(lineSections[1]))
        elif lineSections[0].strip() == "Instructor":
            instructors.append(InstructorProcessor(lineSections[1]))

#print('number of slots: {}, number of rooms: {}, number of courses: {}, number of instructors: {}'.\
#     format(len(slots), len(rooms), len(courses), len(instructors)))


In [ ]:
class courseInstructor:
    def __init__(self, an_item_in_courses, an_item_in_instructors):
        self.course = an_item_in_courses        
        self.instructor = an_item_in_instructors
    
    def printCourseInstructorInfo(self): # Don't forget situation that 2 teachers may have the same name.
        return('course name: {} | instructor name: {}'.\
              format(self.course.getCourseName(),\
                     self.instructor.getInstructorName())) 
    

# Class schedule 

In [ ]:
def IsInstructorAvailable(which_class, CurrentInstructor, ExistedSlots, which_slot):
    if which_class == 1:
        return False
    else:
        i = 1
        a = 0
        while i < which_class:          
            if CurrentInstructor == ExistedSlots[which_slot - 40 * i][0].instructor.getInstructorIndex():
                a += 1
            else:
                a += 0
            
            i += 1
        if a == 0:
            return False
        else:
            return True


In [ ]:
class Schedule:
    def __init__(self, slots, rooms, courses, instructors):
        self.schedule_slots = slots
        self.schedule_rooms = rooms
        self.schedule_courses = courses
        self.schedule_instructors = instructors
        self.slotsInfo = []
        self.slots = []
        self.fitness = 0.0
        self.generation = 1
        self.number_of_courses_for_a_class_per_week = len(self.schedule_slots)
        self.weekday = 5
        self.number_of_courses_for_a_class_per_day=self.number_of_courses_for_a_class_per_week/self.weekday

        for i in range(0, len(slots) * len(rooms)): # range(0, 240)
            self.slots.append([]) #Change self.slots from [] to [[],[],[]...]
            
    def inspectSchedule(self):
        if self.slots[0] == []:
                print('Wake up, before looking up the schedule, create it at first!')
        else:
                
            for i in range(0, len(slots) * len(rooms)):
                print( 'CourseInstructorInfo: {} | {}'\
                      .format(self.slots[i][0].printCourseInstructorInfo(), i))
    # only can be called once        
    def createSchedule(self):
        number_of_courses_for_all_classes_per_week = len(self.slots) # 240
        number_of_course_types = len(self.schedule_courses)//len(self.schedule_rooms) 
        initial_choice = [list(range(0,40)), list(range(40,80)),\
                          list(range(80,120)), list(range(120,160)),\
                          list(range(160,200)), list(range(200,240))]
        
        for course in self.schedule_courses:           
            slotContent = courseInstructor(course, self.schedule_instructors[course.getCourseInstructorIndex() - 1])
            classIndex=(int(course.id)+15)//number_of_course_types
            InstructorIndex = course.getCourseInstructorIndex() # type(classIndex) is float
            repeated_times_for_a_course = int(course.arrangement[1])
            j = 0 
            while j < repeated_times_for_a_course:
                if len(initial_choice[classIndex - 1]) > 1:
                    abondon_choice_1, abondon_choice_2 = random.sample(initial_choice[classIndex - 1], 2) 
                    abondon_choice = abondon_choice_2 if IsInstructorAvailable(classIndex, InstructorIndex, self.slots, abondon_choice_1)\
                    else abondon_choice_1
                else: 
                    abondon_choice = initial_choice[classIndex - 1][0] 
                        
                self.slots[abondon_choice].append(slotContent)
                """
                print('OKAY! The {}th course :{}, place it to the {}th slot'.format(counter_the_number_of_arranged_course,course.getCourseName(), abondon_choice))
                counter_the_number_of_arranged_course += 1
                print(self.slots[abondon_choice][0].printCourseInstructorInfo()) # Different from Version1
                """
                initial_choice[classIndex - 1].remove(abondon_choice)
                j = j + 1    
        
        for k in range(0, len(self.slots)):
            self.slotsInfo.append(self.slots[k][0])
          

## Test class `Schedule` 

In [ ]:
ScheduleTable = Schedule(slots, rooms, courses, instructors )

In [ ]:
len(ScheduleTable.slots)
len(ScheduleTable.slots[0])
ScheduleTable.createSchedule()
len(ScheduleTable.slots[0])
len(ScheduleTable.slots)

In [ ]:
ScheduleTable.slotsInfo[0]

In [ ]:
ScheduleTable.slots[0]
ScheduleTable.slots[0][0]

In [ ]:
begin_time = time.time()
i = 0
while i < 100:
    ScheduleTable.createSchedule()
    i +=1 
cost_time = time.time() -begin_time
cost_time

In [ ]:
ScheduleTable.inspectSchedule()

In [ ]:
#ScheduleTable.mutation()

## Test method of computing of  fitness

<span style = "font-family:Times New Roman ; font-size: 1.5em; color: green"> 
ScheduleTable.slotsInfo = [self.slot[0][0],... self.slot[k][0], ..., self.slot[239][0]]
   
ScheduleTable.slots = [[slotContent], [slotContent]...]  

<span style = "font-family:Times New Roman ; font-size: 1.5em; color: red"> 
Clearly, we can judge the ScheduleTable via withdrawing the data from ScheduleTable.slotsInfo

`self.slots[k][0] is slotContent`

`slotContent = courseInstructor(course, self.schedule_instructors[course.getCourseInstructorIndex() - 1])`


In [ ]:
1%8

In [ ]:
0<=1<2<3

In [ ]:
239%30
239//8

In [ ]:
239//30 # a/b how many b in a

In [ ]:
1//30

### Withdraw the data structure 

Design the data structure for business need a little intelligence, don't take too much time on it.

In [ ]:
courseToBeEvaluated = []
for i in range(0,30):
    courseToBeEvaluated.append([])
    
i = 0
for k in ScheduleTable.slotsInfo:
    m =  i // 8        
    courseToBeEvaluated[m].append(k)
    i += 1

courseToBeEvaluated[0]

In [ ]:
"""
zzz = []
courseNamesSet = []
courseArrangementSet = []
zzzz = [[], []]
for x in courseToBeEvaluated[0]:
    zzz.append({"Course": x.course.name, "Arrangement": x.course.arrangement})
    courseNamesSet.append(x.course.name)
    courseArrangementSet.append(x.course.arrangement)
    zzzz[0].append(x.course.name)
    zzzz[1].append(x.course.arrangement)
    
zzz
courseNamesSet
courseArrangementSet
zzzz 
"""

In [ ]:
z = [[], []]
for x in courseToBeEvaluated[0]:
    z[0].append(x.course.name)
    z[1].append(x.course.arrangement)

z

In [ ]:
ScheduleOfClass = []
x = len(courseToBeEvaluated)
for i in range(x):
    ScheduleOfClass.append([[], []])
ScheduleOfClass

In [ ]:
for i in range(x):
    for j in courseToBeEvaluated[i]:
        ScheduleOfClass[i][0].append(j.course.name)
        ScheduleOfClass[i][1].append(j.course.arrangement)

ScheduleOfClass

In [ ]:
ScheduleOfClass[1]

**Avoid this condition**

In [ ]:
m= dict(Counter(ScheduleOfClass[1][1][0:4]))
m
m['A5']
m['A3']

In [ ]:
m.keys()
list(m.keys())

In [ ]:
'A5' in list(m.keys())
'A5' not in list(m.keys())

In [ ]:
for ooo in z:
    ooo 
    calculateFitness(ooo)

In [ ]:
calculateFitness(ooo)

In [ ]:
m= dict(Counter(ooo[1][0:4]))

m['A5'] 

### Evaluate a course order in a day

Cut`ScheduleTable.slotsInfo`  into slices, A slices represents the courses for a class a day  

In [ ]:
def calculateFitnessForAStrip(courseForADayAClass):
    getMark = 0
    x = len(list(dict(Counter(courseForADayAClass[0])).values()))
    if x == 8:  # Check if the course has no repeated items
        getMark += 5
    elif x == 7:
        getMark += 3
    elif x == 6:# 4+2*2 or 5+3*1
        getMark += 0
    else:
        getMark += -1
    m= dict(Counter(courseForADayAClass[1][0:4]))
    n = dict(Counter(courseForADayAClass[1][4:8]))
    
    if 'A5' in list(m.keys()) and 'A5' not in list(n.keys()):
        if m['A5'] == 2:
            getMark +=3
    if 'A5' not in list(m.keys()) and 'A5' in list(n.keys()):
        if n['A5'] == 2:
            getMark -=3
    else:
        getMark += 0

    return getMark

In [ ]:
def calculateFitness(ScheduleofClass):
    mark = 0
    for ooo in ScheduleOfClass:
        a = calculateFitnessForAStrip(ooo)
        mark += a
    return mark   

In [ ]:
calculateFitness(ScheduleOfClass)

In [ ]:
begin_time = time.time()
i = 0
while i < 100:
    ScheduleTable.createSchedule()
    i +=1 
cost_time = time.time() -begin_time
cost_time

### Create a bunch of ScheduleTable

In [ ]:
chromosomes = []

for i in range(0, 100):
    chromosome =  Schedule(slots, rooms, courses, instructors )
    chromosome.createSchedule()
    chromosomes.append({
        "chromosome": chromosome,
        "fitness": chromosome.calculateFitness()
    })

In [ ]:
i = 0
while i < 1:
    ScheduleTable = Schedule(slots, rooms, courses, instructors)
    ScheduleTable.createSchedule()
    print(ScheduleTable.calculateFitness())
    xxx = ScheduleTable.mutation()
    print(xxx.calculateFitness())
    i += 1

## Mutate a available schedule

In [ ]:
def validMutation(self):
    p4 = copy.deepcopy(self)
    t1 = random.randint(
        0,
        len(self.schedule_rooms) -
        1) * self.number_of_courses_for_a_class_per_week  # t1 <- [0,200)

    classIndex1, classIndex2 = random.sample(
        range(t1, t1 + self.number_of_courses_for_a_class_per_week), 2)
    p4.slotsInfo[classIndex1]["class"], p4.slotsInfo[classIndex2][
        "class"] = p4.slotsInfo[classIndex2]["class"], p4.slotsInfo[
            classIndex1]["class"]
    p4.generation = self.generation + 1
    return p4

# Check if a schedule is available or not

In [ ]:
anAvailableSchedule = ScheduleTable.slotsInfo

In [ ]:
valueOfScheduleTable = ScheduleTable.calculateFitness()

In [ ]:
valueOfScheduleTable 

In [ ]:
def confSlot(anObjectWithAListType):
    conf_Slot = []
    temp = 0
    for k in range(0, 6):
        conf_Slot.append([])
    for k1 in anObjectWithAListType:
        for k2 in anObjectWithAListType:
            if k1["teacher"] == k2["teacher"]:
                conf_Slot[temp].append(k2)
        temp = temp + 1
    return conf_Slot

In [ ]:
def removeDuplication(anObjectWithAListType):
    resultList = []
    for item in anObjectWithAListType:
        if not item in resultList:
            resultList.append(item)
    return resultList

In [ ]:
def confTest(scheduleSlotsInfo): # scheduleSlotsInfo [{},{},{},{},{},{}]
    tempTimeSlotTeacherInfo = []  
    test = []
   
    for i in range(0, 40):
        tempTimeSlotTeacherInfo.append([])
        
    for temp_class in scheduleSlotsInfo: # temp_class {}
        # intention: 
    
        whichSlot = temp_class["slotIndex"] # slotIndex ~ [0, 240)     
        whichTeacherToThisSlot = temp_class["class"].course.instructorIndex
        tempTimeSlotTeacherInfo[whichSlot % 40].append({"teacher": whichTeacherToThisSlot,"time": whichSlot})# Highlight
        
    for tmpTSTI in tempTimeSlotTeacherInfo: # the datatype of tmpTSTI  [{},{},{},{},{},{}]
        t = confSlot(tmpTSTI) # the data type of t [[{},{},...,{}]]
        # Intention: Check confliction
        
        # counter , tmpTSTI[i]["teacher"]
        
        
        
        for j in range(0, 6):
            if len(t[j]) >= 2:
                test.append(t[j])
        test = removeDuplication(test)
    return test

In [ ]:
confTest(anAvailableSchedule)

<span style = "font-family:Times New Roman ; font-size: 1.5em; color: green"> 
What does confTest implement?
</span>

# Class Genetic Algorithm

In [ ]:
class GeneticAlgorithm:
    def __init__(self, slots, rooms, courses, instructors):
        self.slots = slots
        self.rooms = rooms
        self.num_Room=len(self.rooms)
        self.courses = courses
        self.instructors = instructors
        self.week_time=len(slots)
        self.chromosomes = []
        self.newChromosomes = []
        self.newChromosomeCount = 0
        self.lastFitness = 0
        self.fitnessConstantSince = 0
        self.chromosome_num=[]
        self.selectedPairs=[]
        self.ppt_data=[]


    def initChromosomes(self, numberOfChromosomes):
        for i in range(0, numberOfChromosomes):
            chromosome = Schedule(self.slots, self.rooms, self.courses, self.instructors)
            chromosome.createSchedule()
            self.chromosomes.append({"chromosome": chromosome, "fitness": chromosome.calculateFitness()})
            self.chromosome_num.append(i)
        for i in range(0,numberOfChromosomes//2):
            self.selectedPairs.append([])

    def confSlot(self, list):
        conf_Slot=[]
        temp = 0
        for k in range(0,self.num_Room):
            conf_Slot.append([])
        for k1 in list:
            for k2 in list:
               if k1["teacher"]==k2["teacher"]:
                   conf_Slot[temp].append(k2)
            temp=temp+1
        return conf_Slot

    def delete(self, list):
        resultList = []
        for item in list:
            if not item in resultList:
                resultList.append(item)
        return resultList

    #TODO 冲突检测函数
    def confTest(self,sch):
        temptime=[]
        tempteacher=[]
        temptimeteacher=[]
        test=[]
        count=-1
        num=0
        for i in range(0,self.week_time):
            tempteacher.append([])
            temptime.append([])
            temptimeteacher.append([])
        for temp_class in sch:
            temp1Index= temp_class["slotIndex"]
            count=temp1Index % self.week_time
            temp2Index = sch[temp1Index]["class"].course.instructorIndex
            temptimeteacher[temp1Index % self.week_time].append({"teacher":temp2Index,"time":temp1Index})
        for somesch in temptimeteacher: # somesch: [{"teacher": teacher,"time": time}, {}, ..., {}]
            t=self.confSlot(somesch) # t: [[{"teacher": teacher,"time": time}, {}, ..., {}]]
            for j in range(0,self.num_Room):
                if len(t[j])>=2:
                    test.append(t[j])
            test=self.delete(test)
        return test

    #TODO 冲突消除
    def confClear(self,classlist):
        list=self.confTest(classlist)
        for someconf in list:
            x0=random.randint(0,len(someconf)-1)
            x1=someconf[x0]["time"]
            x2=random.randint((x1/self.week_time)*self.week_time,(x1/self.week_time)*self.week_time+self.week_time-1)
            temp=classlist[x1]["class"]
            classlist[x1]["class"]=classlist[x2]["class"]
            classlist[x2]["class"]=temp
        return classlist

    def continueIteration(self):
        for chromosome in self.chromosomes:
            if chromosome["fitness"] ==1:
                L = self.confTest(self.chromosomes[len(self.chromosomes) - 1]["chromosome"].classes)
                print(L)
                while len(L) != 0:
                    self.chromosomes[len(self.chromosomes) - 1]["chromosome"].classes=self.confClear(self.chromosomes[len(self.chromosomes) - 1]["chromosome"].classes)
                    self.chromosomes[len(self.chromosomes) - 1]["fitness"] = self.chromosomes[len(self.chromosomes) - 1]["chromosome"].calculateFitness()
                    L=self.confTest(self.chromosomes[len(self.chromosomes) - 1]["chromosome"].classes)
                print(L)
                print("Best Fitness in Generation: " + str(chromosome["fitness"]))
                chromosome["chromosome"].printObject()
                chromosome["chromosome"].saveHtml('schedule_output.html')
                chromosome["chromosome"].savejson('schedule_output.txt')
                return False

        print(str(self.chromosomes[len(self.chromosomes) - 1]["fitness"]))
        self.ppt_data.append(str(self.chromosomes[len(self.chromosomes) - 1]["fitness"]))
        f = open('ppt_data.txt', 'w')
        f.write(json.dumps(self.ppt_data))
        f.close()
        if self.lastFitness == self.chromosomes[len(self.chromosomes) - 1]["fitness"]:
            self.fitnessConstantSince += 1
            if self.fitnessConstantSince >= 100:
                L = self.confTest(self.chromosomes[len(self.chromosomes) - 1]["chromosome"].classes)
                print(L)
                print(self.chromosomes[len(self.chromosomes) - 1]["chromosome"])
                while len(L) != 0:
                    self.chromosomes[len(self.chromosomes) - 1]["chromosome"].classes = self.confClear(self.chromosomes[len(self.chromosomes) - 1]["chromosome"].classes)
                    self.chromosomes[len(self.chromosomes) - 1]["fitness"] = self.chromosomes[len(self.chromosomes) - 1]["chromosome"].calculateFitness()
                    L = self.confTest(self.chromosomes[len(self.chromosomes) - 1]["chromosome"].classes)
                print(L)

                print(str(self.chromosomes[len(self.chromosomes) - 1]["fitness"]))
                self.chromosomes[len(self.chromosomes) - 1]["chromosome"].printObject()
                self.chromosomes[len(self.chromosomes) - 1]["chromosome"].saveHtml('schedule_output.html')
                chromosome["chromosome"].savejson('schedule_output.txt')
                print("Fitness score does not seem improve anymore. This schedule may not satify some requirements but a \"significant\" time is passed, so it is returned. Please try again with (smaller) inputs")
                return False
        else:
            self.lastFitness = self.chromosomes[len(self.chromosomes) - 1]["fitness"]
            self.fitnessConstantSince = 0
        return True

    def execute(self):
        while self.continueIteration():

            self.newChromosomes = []
            for mutationIndex in range(len(self.chromosomes) // 2, len(self.chromosomes)):
                if random.randint(0, 4) == 1:
                    # mutation probablity 25%
                    p4 = self.chromosomes[mutationIndex]["chromosome"].mutation()
                    self.newChromosomes.append({"chromosome": p4, "fitness": p4.calculateFitness()})
            
            print(len(self.newChromosomes))
            self.chromosomes = sorted(self.chromosomes, key=itemgetter('fitness'))

            for i in range(0, min(len(self.newChromosomes), int(len(self.chromosomes) * 0.8))): # keep the top 20% chromosomes
                L1 = self.confTest(self.newChromosomes[i]["chromosome"].classes)
                while len(L1) != 0: # until L1 == 0, stop executing this block, that means, the newChrosomes has no confliction.
                    self.newChromosomes[i]["chromosome"].classes = self.confClear(self.newChromosomes[i]["chromosome"].classes)
                    self.newChromosomes[i]["fitness"] = self.newChromosomes[i]["chromosome"].calculateFitness()
                    L1 = self.confTest(self.newChromosomes[i]["chromosome"].classes)
                self.chromosomes[i] = self.newChromosomes[i]
